# Training a simple CNN model in Tensorflow for Tornado Detection

This notebook steps through how to train a simple CNN model using a subset of TorNet.

This will not produce a model with any skill, but simply provides a working end-to-end example of how to set up a data loader, build, and fit a model


In [1]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
#sys.path.append('../')  

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tornet.data.tf.loader import create_tf_dataset 
from tornet.data.constants import ALL_VARIABLES

2024-01-25 15:37:38.337887: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-25 15:37:38.340166: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-25 15:37:38.385485: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-25 15:37:38.386856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-25 15:37:39.216553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Create basic dataloader
# This option loads directly from netcdf files, and will be slow and IO bound
# To speed up training, either
#     build as a tensorflow_dataset , (see tornet/data/tfds/tornet/README.md)
#     cache dataset first , or
#     use tf.data.Dataset.load on a pre-saved dataset

# Location of tornet
data_root = os.environ['TORNET_ROOT']

# Get training data from 2018
data_type='train'
years = [2018,]

catalog_path = os.path.join(data_root,'catalog.csv')
if not os.path.exists(catalog_path):
    raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type']==data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1,random_state=1234)
file_list = [os.path.join(data_root,f) for f in catalog.filename]

ds = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1) 

# (Optional) Save data for faster reloads (makes copy of data!)
# ds.save('tornet_sample.tfdataset') 


In [3]:
# If saved with ds.save(...), just load that model
#ds = tf.data.Dataset.load('tornet_sample.tfdataset')

In [4]:
# If data was registered in tensorflow_dataset, use that
# env variable TFDS_DATA_DIR should point to location of this resaved dataset
#import tensorflow_datasets as tfds
#import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'

#data_type='train'
#years = [2018,]
#ds = tfds.load('tornet',split='+'.join(['%s-%d' % (data_type,y) for y in years]))

In [5]:
import tornet.data.preprocess as pp
from tornet.data.tf import preprocess as tfpp

# Preprocess

# add 'coordinates' variable used by CoordConv layers
ds = ds.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
     
# Take only last time frame
ds = ds.map(pp.remove_time_dim)

# Split sample into inputs,label
ds = ds.map(tfpp.split_x_y)

# (Optional) add sample weights
# weights={'wN':1.0,'w0':1.0,'w1':1.0,'w2':2.0,'wW':0.5}
# ds = ds.map(lambda x,y:  tfpp.compute_sample_weight(x,y,**weights) )

ds = ds.prefetch(tf.data.AUTOTUNE)
        
ds = ds.batch(32)



In [6]:
# Create a simple CNN model
# This normalizes data, concatenates along channel, and applies a Conv2D
import keras
from tornet.data.constants import CHANNEL_MIN_MAX

input_vars = ALL_VARIABLES # which variables to use

# TF convention is B,L,W,H
inputs = {v:keras.Input(shape=(120,240,2),name=v) for v in input_vars}

# Normalize inputs
norm_layers = []
for v in input_vars:
    min_max = np.array(CHANNEL_MIN_MAX[v]) # [2,]

    # choose mean,var to get approximate [-1,1] scaling
    var=((min_max[1]-min_max[0])/2)**2 # scalar
    var=np.array(2*[var,])    # [n_sweeps,]
    offset=(min_max[0]+min_max[1])/2    # scalar
    offset=np.array(2*[offset,]) # [n_sweeps,]
    
    norm_layers.append(
        keras.layers.Normalization(mean=offset, variance=var,
                                   name='Normalized_%s' % v)
    )

# Concatenate normed inputs along channel dimension
x=keras.layers.Concatenate(axis=-1,name='Concatenate1')(
        [l(inputs[v]) for l,v in zip(norm_layers,input_vars)]
        )

# Replace background (nan) with -3
x=keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x),-3.0,x),name='ReplaceNan')(x)

# Processing
x = keras.layers.Conv2D(32,3,padding='same',activation='relu')(x)
# add more..
x = keras.layers.Conv2D(1,1,padding='same',activation='relu',name='TornadoLikelihood')(x)
y = keras.layers.GlobalMaxPool2D(name='GlobalMaxPool')(x)

model = keras.Model(inputs=inputs,outputs=y,name='TornadoDetector')

model.summary()

Model: "TornadoDetector"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 DBZ (InputLayer)            [(None, 120, 240, 2)]        0         []                            
                                                                                                  
 VEL (InputLayer)            [(None, 120, 240, 2)]        0         []                            
                                                                                                  
 KDP (InputLayer)            [(None, 120, 240, 2)]        0         []                            
                                                                                                  
 RHOHV (InputLayer)          [(None, 120, 240, 2)]        0         []                            
                                                                                    

In [7]:
# Compile
opt  = keras.optimizers.Adam(learning_rate=1e-3)
loss=keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

In [8]:
# Train


# steps_per_epoch=10 for demo purposes
model.fit(ds,epochs=3,steps_per_epoch=10)

Epoch 1/3


/state/partition1/llgrid/pkg/anaconda/anaconda3-2023b/lib/python3.9/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['range_folded_mask', 'category', 'event_id', 'ef_number', 'az_lower', 'az_upper', 'rng_lower', 'rng_upper', 'time', 'coordinates'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


10/10 [==============================] - 17s 2s/step - loss: 1.2716
Epoch 2/3
10/10 [==============================] - 16s 2s/step - loss: 0.7283
Epoch 3/3
10/10 [==============================] - 16s 2s/step - loss: 0.6942


In [9]:
# Build a test set
# Basic loader
data_type='test'
years = [2018]

catalog_path = os.path.join(data_root,'catalog.csv')
if not os.path.exists(catalog_path):
    raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type']==data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1,random_state=1234)
file_list = [os.path.join(data_root,f) for f in catalog.filename]

ds_test = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1) 


In [10]:
# TFDS loader
# env variable TFDS_DATA_DIR should point to location of resaved dataset
#import tensorflow_datasets as tfds
#import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'

#data_type='test'
#years = [2018,]
#ds_test = tfds.load('tornet',split='+'.join(['%s-%d' % (data_type,y) for y in years]))

In [11]:
# preprocess
ds_test = ds_test.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
ds_test = ds_test.map(pp.remove_time_dim)
ds_test = ds_test.map(tfpp.split_x_y)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)    
ds_test = ds_test.batch(32)


In [12]:
# Evaluate
import tornet.metrics.tf.metrics as km
metrics = [km.AUC(from_logits=True,name='AUC'),
           km.BinaryAccuracy(from_logits=True,name='BinaryAccuracy'), 
           ]
model.compile(loss=loss,metrics=metrics)

# steps=10 for demo purposes
model.evaluate(ds_test,steps=10)


/state/partition1/llgrid/pkg/anaconda/anaconda3-2023b/lib/python3.9/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['range_folded_mask', 'category', 'event_id', 'ef_number', 'az_lower', 'az_upper', 'rng_lower', 'rng_upper', 'time', 'coordinates'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


10/10 [==============================] - 17s 2s/step - loss: 0.6938 - AUC: 0.5249 - BinaryAccuracy: 0.9406


[0.6937962770462036, 0.524876594543457, 0.940625011920929]